<a href="https://colab.research.google.com/github/PhebeJ/Project/blob/main/IRIS_LOCALIZATION_and_NORMALIZATION_for_image1_folder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IRIS LOCALIZATION AND NORMALIZATION**


---



In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


### **IRIS LOCALIZATION**

---



In [2]:
import os
import warnings

In [3]:
import cv2
import numpy as np

from skimage import transform, feature

In [4]:
def read_imgs():
    """read all images of CASIA Iris Image Database (version 1.0)
    :return: the train set and test set
    :rtype: tuple (train, test)
    """

    base_dir = '/gdrive/MyDrive/Images'
    classes = os.listdir(base_dir)
    train, test = [], []
    for c in classes:
        tr_dir = '%s/%s/1/' % (base_dir, c)
        te_dir = '%s/%s/2/' % (base_dir, c)
        for f in os.listdir(tr_dir):
            if f[-3:] == 'bmp':
                train.append(cv2.imread(tr_dir + f, 0))
        for f in os.listdir(te_dir):
            if f[-3:] == 'bmp':
                test.append(cv2.imread(te_dir + f, 0))
    print("Training output:", train)
    return train, test

### **IRIS NORMALIZATION**

---



In [5]:
#NORMALIZATION
import math

import numpy as np

def iris_normalization(img, pupil_circle, iris_circle, M=64, N=512, offset=0):
    """normalize the iris.
    :param img: the input img
    :param pupil_circle: (x, y, radius)
    :param iris_circle: (x, y, radius)
    :param M, N: the normalization image size
    :param offset: the initial angle
    :return: the normalization image
    :rtype: ndarray
    """

    normalized = np.zeros((M, N))
    theta = np.linspace(0, 2 * np.pi, N)

    for i in range(N):
        curr_theta = theta[i] + offset
        if curr_theta > 2 * np.pi:
            curr_theta -= 2 * np.pi
        begin = trans_axis(pupil_circle, curr_theta)
        end = trans_axis(iris_circle, curr_theta)

        xspace = np.linspace(begin[0], end[0], M)
        yspace = np.linspace(begin[1], end[1], M)
        normalized[:, i] = [255 - img[int(y), int(x)]
                            if 0 <= int(x) < img.shape[1] and 0 <= int(y) < img.shape[0]
                            else 0
                            for x, y in zip(xspace, yspace)]
    return normalized

def trans_axis(circle, theta):
    """Changes polar coordinates to cartesian coordinate system.
    :param circle: (x, y, radius)
    :param theta: angle
    :return: new coordinates (x, y)
    :rtype: tuple (int, int)
    """

    x0, y0, r = circle
    x = int(x0 + r * math.cos(theta))
    y = int(y0 + r * math.sin(theta))
    return x, y


### **IMAGE ENHANCEMENT**

In [6]:
import cv2
import numpy as np

from skimage.filters import rank
import skimage.morphology as morp

def enhance_img(img):
    """actually, the enhance method is based on another Ma Li's paper.
        'Iris Recognition Based on Multichannel Gabor Filtering'
    :param img: the input img
    :return: the enhanced image
    :rtype: ndarray
    """
    kernel = morp.disk(32)
    img_local = rank.equalize(img.astype(np.uint8), selem=kernel)

    enhanced = cv2.GaussianBlur(img_local, (5, 5), 0)
    return enhanced



---



In [7]:
def process_imgs(imgs, use_offset=False):
    """process the input raw images to feature vectors
    :imgs: the image set
    :use_offset: in the paper, it denotes unwrapping the iris with different angles can
    remove rotation invariance. However, it seems no effect.
    :return: the train set and test set
    :rtype: tuple (train, test)
    """

    processed = []
    for img in imgs:
        circles = detect_by_hough(img)
        circles = np.array(circles).reshape(1 ,2 ,3)

        # denoising
        (_, B) = cv2.threshold(img ,180 ,255 ,cv2.THRESH_BINARY)
        (_, C) = cv2.threshold(img ,100 ,255 ,cv2.THRESH_BINARY)
        img = img & ~B & C

        if use_offset:
            offsets = [-9 ,-6 ,-3 ,0 ,3 ,6 ,9]

            for offset in offsets:
                normalized = iris_normalization(img,
                                                circles[0][0], circles[0][1],
                                                offset=offset)
                enhanced = enhance_img(normalized)

                ROI = enhanced[0:48]
                print("ROI is:", ROI)
                filtered_1, _ = defined_gabor(ROI, frequency=0.1, sigma_x=3, sigma_y=1.5)
                print("filtered_1 is:", filtered_1)
                filtered_2, _ = defined_gabor(ROI, frequency=0.07, sigma_x=4.5, sigma_y=1.5)
                print("filtered_2 is", filtered_2)
                feature_vector = get_feature_vector(filtered_1, filtered_2)
                print("feature_vector is", feature_vector)
                processed.append(feature_vector)

        else:
            normalized = iris_normalization(img, circles[0][0], circles[0][1])
            enhanced = enhance_img(normalized)
            ROI = enhanced[0:48]
            print("ROI is:", ROI)
            filtered_1, _ = defined_gabor(ROI, frequency= 32 *np. pi /180, sigma_x=3, sigma_y=1.5)
            print("filtered_1 is:", filtered_1)
            filtered_2, _ = defined_gabor(ROI, frequency= 32 *np. pi /180, sigma_x=4.5, sigma_y=1.5)
            print("filtered_2 is", filtered_2)
            feature_vector = get_feature_vector(filtered_1, filtered_2)
            print("feature_vector is", feature_vector)
            processed.append(feature_vector)

        print ('processed imgs: %i/%i' % (len(processed), len(imgs)), end='\r')

    return ROI


In [8]:
def main():
    train_imgs, test_imgs = read_imgs()

    train = process_imgs(train_imgs, use_offset=False)
    train_labels = np.repeat(range(108), 3)
    print("train_labels are:", train_labels)

In [9]:
def detect_by_hough(img):
    """preprocess the image, then use hough transform to detect both pupil and iris.
    :param img: the input image
    :return: the circles of pupil and iris, (x, y, radius), (x, y, radius)
    :rtype: tuple
    """

    #I have used the roughly_localize method, since it is error-prone.

    def preprocess(img, pupil=False):
        # respectively process pupil and iris
        if pupil:
            thresh = cv2.adaptiveThreshold(img, 255,
                                           cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           cv2.THRESH_BINARY, 5, 3)
            blur = cv2.GaussianBlur(thresh, (9, 9), 0)
            blur = cv2.medianBlur(blur, 33)
            #
            canny = cv2.Canny(blur, 15, 50)
        else:
            blured = img.copy()
            for i in range(3):
                blured = cv2.medianBlur(blured, 11)
            canny = cv2.Canny(blured, 15, 30)
            canny[:,
            pupil_circle[0] - pupil_circle[2] - 30:pupil_circle[0] + pupil_circle[2] + 30] = 0
            canny[0:pupil_circle[1] - pupil_circle[2]] = 0

        return canny

    canny = preprocess(img, pupil=True)

    pupil_circle = customed_hough_circle(canny, hough_radii=range(30, 70, 5))

    # slightly enlarge the radius of pupil
    pupil_circle[2] += 10

    canny = preprocess(img)

    iris_circle = customed_hough_circle(canny, hough_radii=range(pupil_circle[2] + 50, 150, 5))  # circles[0][0]

    # if the distance of iris center and pupil center is too far, we fix the iris center.
    if ((iris_circle - pupil_circle)[:2] ** 2).sum() ** 0.5 > pupil_circle[-1] * 0.3:
        iris_circle[:2] = pupil_circle[:2]

    return pupil_circle, iris_circle


def get_pupil_roughly(img, binarize=False):
    """roughly localize pupil, using the method introduced in the paper.
    :param img: the input image
    :param binarize: whether binarize the img firstly
    :return: the center coordinates (x, y)
    :rtype: tuple (int, int)
    """

    if binarize:
        (_, img) = cv2.threshold(img,
                                 0, 255,
                                 cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    x_min = img.sum(axis=0).argmin()
    y_min = img.sum(axis=1).argmin()

    return x_min, y_min


def select_region(img, x_min, y_min, size=60):
    """select a square  region centered at (x_min, y_min).
    :param img: the input image
    :param x_min, y_min: the center coordinates
    :param size: the range
    :return: the selected image
    :rtype: ndarray
    """

    size = min(x_min, y_min, size)
    return img[y_min - size:y_min + size, x_min - size:x_min + size]


def roughly_localize(img, size=60):
    """the whole procedure of roughly localizing pupil.
    :param img: the input image
    :param size: the range
    :return: the center coordinates (x, y)
    :rtype: tuple (int, int)
    """

    x_min, y_min = get_pupil_roughly(img)

    # the coordinates should not be off center too much.
    if abs(x_min - 160) > 30:
        x_min = 160
    if abs(y_min - 140) > 30:
        y_min = 140

    # localize 2 times.
    for i in range(2):
        x, y = get_pupil_roughly(select_region(img, x_min, y_min, size=size),
                                 binarize=True)
        if abs(x_min - 160) > 30 or abs(y_min - 140) > 30:
            break
        x_min -= size - x
        y_min -= size - y

    return x_min, y_min


def customed_hough_circle(img, hough_radii=range(30, 60)):
    """find circles on given img.
    :param img: the input image
    :param hough_radii: the radii of candidate circles
    :return: the best circle, (x, y, radius)
    :rtype: tuple (int, int, int)
    """

    hough_res = transform.hough_circle(img, hough_radii)
    centers = []
    accums = []
    radii = []

    for radius, h in zip(hough_radii, hough_res):
        num_peaks = 10
        peaks = feature.peak_local_max(h, num_peaks=num_peaks)
        centers.extend(peaks)
        accums.extend(h[peaks[:, 0], peaks[:, 1]])
        radii.extend([radius] * num_peaks)
    best = np.argsort(accums)[::-1][0]
    return np.array([centers[best][1], centers[best][0], radii[best]])

In [10]:

import numpy as np

from scipy import ndimage as ndi


from skimage.util import view_as_blocks

def get_feature_vector(filtered_1, filtered_2):
    """As the paper denotes, this method generate the feature vector
    based on the two filtered image.
    :param filtered_1: the filtered image 1
    :param filtered_2: the filtered image 2
    :return: the feature vector
    :rtype: ndarray
    """

    blocks_1 = view_as_blocks(filtered_1, block_shape=(8, 8)).reshape([-1, 64])
    blocks_2 = view_as_blocks(filtered_2, block_shape=(8, 8)).reshape([-1, 64])

    def mad(array, axis):
        return np.mean(np.abs(array - np.mean(array, axis, keepdims=True)), axis)

    m_1 = blocks_1.mean(axis=-1)
    m_2 = blocks_2.mean(axis=-1)
    mad_1 = mad(blocks_1, axis=-1)
    mad_2 = mad(blocks_2, axis=-1)

    #     feature_vector = np.concatenate([np.stack([m_1, mad_1], axis=1).reshape([-1]),
    #                                     np.stack([m_2, mad_2], axis=1).reshape([-1])])
    feature_vector = np.stack([m_1, mad_1, m_2, mad_2], axis=1).reshape([-1])
    print("Feature Vector inside get_feature_vector:", feature_vector)
    return feature_vector

def defined_gabor_kernel(frequency, sigma_x=None, sigma_y=None,
                         n_stds=3, offset=0, theta=0):
    """
    According to the codes of skimage, I directly rewrote the function gabor_kernel.

    Parameters
    ----------
    frequency : float
        Spatial frequency of the harmonic function. Specified in pixels.
    theta : float, optional
        Orientation in radians. If 0, the harmonic is in the x-direction.
    sigma_x, sigma_y : float, optional
        Standard deviation in x- and y-directions. These directions apply to
        the kernel *before* rotation. If `theta = pi/2`, then the kernel is
        rotated 90 degrees so that `sigma_x` controls the *vertical* direction.
    n_stds : scalar, optional
        The linear size of the kernel is n_stds (3 by default) standard
        deviations
    offset : float, optional
        Phase offset of harmonic function in radians.
    Returns
    -------
    g : complex array
        Complex filter kernel.
    """

    x0 = np.ceil(max(np.abs(n_stds * sigma_x * np.cos(theta)),
                     np.abs(n_stds * sigma_y * np.sin(theta)), 1))
    y0 = np.ceil(max(np.abs(n_stds * sigma_y * np.cos(theta)),
                     np.abs(n_stds * sigma_x * np.sin(theta)), 1))
    y, x = np.mgrid[-y0:y0 + 1, -x0:x0 + 1]

    g = np.zeros(y.shape, dtype=np.complex)
    g[:] = np.exp(-0.5 * (x ** 2 / sigma_x ** 2 + y ** 2 / sigma_y ** 2))
    g /= 2 * np.pi * sigma_x * sigma_y
    g *= np.cos(2 * np.pi * frequency * ((x ** 2 + y ** 2) ** 0.5))
    print("g is:", g)
    return g


def defined_gabor(img, frequency, sigma_x, sigma_y):
    """
    Perform gabor filter on the image using defined kernel.

    Parameters
    ----------
    param img : the input img
    frequency : float
        Spatial frequency of the harmonic function. Specified in pixels.
    sigma_x, sigma_y : float, optional
        Standard deviation in x- and y-directions. These directions apply to
        the kernel *before* rotation. If `theta = pi/2`, then the kernel is
        rotated 90 degrees so that `sigma_x` controls the *vertical* direction.
    -------
    Returns   :
    filtered_real, filtered_imag : the filtered image. Because using the evensymmetric
    filter, the filtered_imag is zero.
    """

    g = defined_gabor_kernel(frequency, sigma_x, sigma_y)
    filtered_real = ndi.convolve(img, np.real(g), mode='wrap', cval=0)
    filtered_imag = ndi.convolve(img, np.imag(g), mode='wrap', cval=0)
    print("filtered_real is:", filtered_real)
    print("filtered_imag is:", filtered_imag)
    return filtered_real, filtered_imag



In [11]:
if __name__ =='__main__':
    warnings.filterwarnings("ignore")
    main()

Training output: [array([[176, 166, 170, ..., 187, 180, 169],
       [170, 169, 173, ..., 185, 181, 179],
       [177, 165, 173, ..., 198, 187, 169],
       ...,
       [154, 155, 155, ..., 169, 178, 199],
       [159, 148, 154, ..., 170, 177, 192],
       [153, 154, 150, ..., 172, 171, 189]], dtype=uint8), array([[193, 180, 187, ..., 186, 183, 185],
       [180, 180, 180, ..., 185, 187, 178],
       [182, 186, 178, ..., 193, 184, 176],
       ...,
       [167, 163, 167, ..., 180, 185, 186],
       [170, 165, 154, ..., 178, 182, 198],
       [165, 166, 170, ..., 186, 205, 189]], dtype=uint8), array([[158, 164, 165, ..., 180, 185, 180],
       [168, 154, 169, ..., 184, 185, 178],
       [161, 162, 170, ..., 191, 177, 179],
       ...,
       [145, 142, 147, ..., 177, 174, 172],
       [137, 144, 142, ..., 186, 167, 181],
       [131, 141, 148, ..., 175, 174, 177]], dtype=uint8), array([[182, 176, 195, ..., 182, 178, 178],
       [185, 197, 181, ..., 174, 176, 178],
       [188, 190, 186